# 00. Environment Validation

Este notebook valida el entorno Python y la configuración hardware/software para el laboratorio de aprendizaje federado. Se comprueba la versión de Python, los paquetes principales y el soporte de aceleración hardware (CPU, GPU, NPU).

---

## 1. Versión de Python y ruta del intérprete

Comprobamos la versión de Python y la ruta del intérprete que está usando el entorno.

In [1]:
import sys

print(f"Versión de Python: {sys.version}")
print(f"Ruta del intérprete: {sys.executable}")

Versión de Python: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:20:19) [MSC v.1943 64 bit (AMD64)]
Ruta del intérprete: C:\Users\padul\anaconda3\envs\ia-flower\python.exe


## 2. Versiones de los paquetes principales

Listamos y mostramos las versiones de los paquetes clave para el laboratorio.

In [2]:
import importlib

packages = [
    'numpy', 'pandas', 'matplotlib', 'psutil', 'tqdm',
    'torch', 'flwr', 'openvino', 'openvino_genai', 'notebook'
]

for pkg in packages:
    try:
        mod = importlib.import_module(pkg)
        ver = getattr(mod, '__version__', 'unknown')
        print(f"{pkg}: {ver}")
    except Exception as e:
        print(f"{pkg}: NO INSTALADO ({e})")

numpy: 2.3.2
pandas: 2.3.1
matplotlib: 3.10.3
psutil: 7.0.0
tqdm: 4.67.1
torch: 2.5.1
flwr: 1.19.0
openvino: 2025.2.0-19140-c01cd93e24d-releases/2025/2
openvino_genai: 2025.2.0.0-2260-01f0fe1eded
notebook: 7.4.4


## 3. Validación de GPU NVIDIA y versión de CUDA con PyTorch

Verificamos si PyTorch detecta la GPU NVIDIA y mostramos la versión de CUDA.

In [3]:
import torch

gpu_available = torch.cuda.is_available()
print(f"¿GPU disponible (torch.cuda.is_available)? {gpu_available}")
if gpu_available:
    print(f"Versión de CUDA (torch.version.cuda): {torch.version.cuda}")
    print(f"Número de GPUs: {torch.cuda.device_count()}")
    print(f"Nombre de la GPU: {torch.cuda.get_device_name(0)}")
else:
    print("PyTorch no detecta una GPU NVIDIA.")

¿GPU disponible (torch.cuda.is_available)? True
Versión de CUDA (torch.version.cuda): 12.6
Número de GPUs: 1
Nombre de la GPU: NVIDIA GeForce RTX 4080 Laptop GPU


## 4. Detección de dispositivos con OpenVINO

Listamos los dispositivos disponibles (CPU, GPU, NPU) usando OpenVINO.

In [4]:
try:
    from openvino import Core

    core = Core()
    devices = core.available_devices
    print("Dispositivos disponibles en OpenVINO:")
    for d in devices:
        print(f" - {d}")
    print("\nResumen de dispositivos:")
    print(f"CPU detectado: {any('CPU' in d for d in devices)}")
    print(f"GPU detectado: {any('GPU' in d for d in devices)}")
    print(f"NPU (Intel AI Boost) detectado: {any('NPU' in d or 'AUTO:NPU' in d for d in devices)}")
except Exception as e:
    print(f"Fallo en la detección de dispositivos OpenVINO: {e}")

Dispositivos disponibles en OpenVINO:
 - CPU
 - GPU
 - NPU

Resumen de dispositivos:
CPU detectado: True
GPU detectado: True
NPU (Intel AI Boost) detectado: True


## 5. Validación de scikit-learn

Validamos la instalación y configuración de scikit-learn, incluyendo versión, módulos principales y configuración detallada.

In [5]:
print("\n--- Validando scikit-learn ---")
try:
    import sklearn

    print(f"✔ scikit-learn importado correctamente. Versión: {sklearn.__version__}")
    # Verificar algunos módulos principales
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    print("✔ Módulos principales de scikit-learn disponibles")
    # Mostrar configuración detallada
    sklearn.show_versions()
except Exception as e:
    print(f"✗ Error al validar scikit-learn: {e}")


--- Validando scikit-learn ---
✔ scikit-learn importado correctamente. Versión: 1.7.1
✔ Módulos principales de scikit-learn disponibles

System:
    python: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:20:19) [MSC v.1943 64 bit (AMD64)]
executable: C:\Users\padul\anaconda3\envs\ia-flower\python.exe
   machine: Windows-11-10.0.26100-SP0

Python dependencies:
      sklearn: 1.7.1
          pip: 25.1.1
   setuptools: 75.8.2
        numpy: 2.3.2
        scipy: 1.16.0
       Cython: None
       pandas: 2.3.1
   matplotlib: 3.10.3
       joblib: 1.5.1
threadpoolctl: 3.6.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: mkl
    num_threads: 1
         prefix: libblas
       filepath: C:\Users\padul\anaconda3\envs\ia-flower\Library\bin\libblas.dll
        version: 2025.2-Product
threading_layer: intel

       user_api: openmp
   internal_api: openmp
    num_threads: 1
         prefix: libiomp
       filepath: C:\Users\padul\anaconda3\envs\ia-f

## 6. Resumen de validación del entorno

Esta celda resume la validación. Si falta algún componente crítico, revisa la salida anterior y realiza la instalación o configuración necesaria.

In [6]:
# Resumen simple del entorno
missing = []

# Comprobación de versión de Python
if sys.version_info < (3, 8):
    missing.append('Se requiere Python >= 3.8')

# Comprobación de paquetes
for pkg in packages:
    try:
        importlib.import_module(pkg)
    except:
        missing.append(f"{pkg} no está instalado")

# Comprobación de GPU con PyTorch
if not torch.cuda.is_available():
    missing.append('PyTorch no detecta GPU NVIDIA')

# Comprobación de dispositivos OpenVINO
try:
    ov_devices = core.available_devices
    if not any('CPU' in d for d in ov_devices):
        missing.append('OpenVINO no detecta CPU')
    if not any('GPU' in d for d in ov_devices):
        missing.append('OpenVINO no detecta GPU')
    if not any('NPU' in d or 'AUTO:NPU' in d for d in ov_devices):
        missing.append('OpenVINO no detecta NPU (Intel AI Boost)')
except:
    missing.append('Fallo en la detección de dispositivos OpenVINO')

# Comprobación de scikit-learn
try:
    import sklearn
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    sklearn.show_versions()
except Exception:
    missing.append('scikit-learn no está correctamente instalado o configurado')

if missing:
    print('El entorno NO está listo. Problemas detectados:')
    for m in missing:
        print(f'- {m}')
else:
    print('¡El entorno está LISTO para los experimentos de aprendizaje federado!')


System:
    python: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:20:19) [MSC v.1943 64 bit (AMD64)]
executable: C:\Users\padul\anaconda3\envs\ia-flower\python.exe
   machine: Windows-11-10.0.26100-SP0

Python dependencies:
      sklearn: 1.7.1
          pip: 25.1.1
   setuptools: 75.8.2
        numpy: 2.3.2
        scipy: 1.16.0
       Cython: None
       pandas: 2.3.1
   matplotlib: 3.10.3
       joblib: 1.5.1
threadpoolctl: 3.6.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: mkl
    num_threads: 1
         prefix: libblas
       filepath: C:\Users\padul\anaconda3\envs\ia-flower\Library\bin\libblas.dll
        version: 2025.2-Product
threading_layer: intel

       user_api: openmp
   internal_api: openmp
    num_threads: 1
         prefix: libiomp
       filepath: C:\Users\padul\anaconda3\envs\ia-flower\Library\bin\libiomp5md.dll
        version: None

       user_api: openmp
   internal_api: openmp
    num_threads: 22
         pref